# SEC Form 4 Filings: Who is Buying and Who is Selling?

Insider transaction reporting via SEC Form 4 plays an important role in market transparency by forcing corporate officers, directors, and large shareholders to disclose their stock trades within two business days of execution. Because Form 4s contain rich, structured information—transaction dates, security details, number of shares, prices, and the insider’s relationship to the issuer—systematically extracting and aggregating these filings enables analysts and investors to detect patterns such as confidence buys, opportunistic sales, or hedging activity long before quarterly disclosures arrive. 

Automating Form 4 extraction addresses challenges of inconsistent XML/TXT layouts, varying footnote structures, and high filing volumes, ensuring that trading signals derived from insider behavior are captured accurately, updated in real time, and integrated seamlessly into quantitative models and trading dashboards. While it is possible to manually extract from the XML data, this can be a time consuming and error prone process. We will show how we can use LlamaExtract to do this automatically without having to parse the XML data.

## Dow Jones Industrial Average Companies

Dow Jones Industrial Average (DJIA) is a stock market index that consists of 30 large companies listed on the New York Stock Exchange and the NASDAQ and is considered a good proxy for the overall US stock market. For this exercise, we will extract the insider transactions for all the companies in the DJIA. Let's first get the list of tickers in the Dow Jones Industrial Average using Wikipedia.

In [ ]:
%pip install pandas
%pip install lxml


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd

url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
parsed = pd.read_html(url, header=0)
dow = next(t for t in parsed if "Symbol" in t.columns)
DJIA_TICKERS = dow["Symbol"].tolist()
DJIA_TICKERS

['MMM',
 'AXP',
 'AMGN',
 'AMZN',
 'AAPL',
 'BA',
 'CAT',
 'CVX',
 'CSCO',
 'KO',
 'DIS',
 'GS',
 'HD',
 'HON',
 'IBM',
 'JNJ',
 'JPM',
 'MCD',
 'MRK',
 'MSFT',
 'NKE',
 'NVDA',
 'PG',
 'CRM',
 'SHW',
 'TRV',
 'UNH',
 'VZ',
 'V',
 'WMT']

Let's see an example of a Form 4 filing for MMM. Disregard how we got the URL below, we will show how to get the URL for all the Form 4 filings in the utility function below.

🖋: We are using `hishel` to cache the responses from the SEC website to avoid being rate limited or blocked.

In [ ]:
%pip install hishel


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import httpx
import hishel
import os


# Change this to the relevant output directory
DATA_DIR = "./data/insider_transactions"
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
CACHE_DIR = os.path.join(DATA_DIR, ".httpx_cache")
FORM_4_DUMP_PATH = os.path.join(DATA_DIR, "sec_form_4_dump.json")


transport = hishel.AsyncCacheTransport(
    transport=httpx.AsyncHTTPTransport(),
    storage=hishel.AsyncFileStorage(base_path=CACHE_DIR),
    controller=hishel.Controller(force_cache=True),
)

# Use your own email below
HEADERS = {"User-Agent": "Your Name your_email@domain.com"}
http_client = httpx.AsyncClient(headers=HEADERS, timeout=60, transport=transport)
example_url = "https://www.sec.gov/Archives/edgar/data/66740/000112760225005147/0001127602-25-005147.txt"
r = await http_client.get(example_url)
print(r.text)

<SEC-DOCUMENT>0001127602-25-005147.txt : 20250218
<SEC-HEADER>0001127602-25-005147.hdr.sgml : 20250218
<ACCEPTANCE-DATETIME>20250218111341
ACCESSION NUMBER:		0001127602-25-005147
CONFORMED SUBMISSION TYPE:	4
PUBLIC DOCUMENT COUNT:		1
CONFORMED PERIOD OF REPORT:	20250207
FILED AS OF DATE:		20250218
DATE AS OF CHANGE:		20250218

REPORTING-OWNER:	

	OWNER DATA:	
		COMPANY CONFORMED NAME:			Chavez Rodriguez Beatriz Karina
		CENTRAL INDEX KEY:			0001890011
		ORGANIZATION NAME:           	

	FILING VALUES:
		FORM TYPE:		4
		SEC ACT:		1934 Act
		SEC FILE NUMBER:	001-03285
		FILM NUMBER:		25633441

	MAIL ADDRESS:	
		STREET 1:		3M COMPANY
		STREET 2:		3M CENTER BLDG 220-09-E-02
		CITY:			ST. PAUL
		STATE:			MN
		ZIP:			55144-1000

ISSUER:		

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			3M CO
		CENTRAL INDEX KEY:			0000066740
		STANDARD INDUSTRIAL CLASSIFICATION:	SURGICAL & MEDICAL INSTRUMENTS & APPARATUS [3841]
		ORGANIZATION NAME:           	08 Industrial Applications and Services
		IRS NUMBER:

## Data Schema for Form 4 Extraction

Let's define the data schema for the Form 4 extraction. We will use Pydantic to define our schema, and use LlamaExtract to extract the data from the Form 4 filings. Note that since we are using an LLM to extract the data, we don't need to worry about the exact layout/schema for the XML files. 

In [ ]:
from pydantic import BaseModel, Field
from typing import Dict, List, Optional


class Address(BaseModel):
    street1: str = Field(description="Business or owner street address line 1")
    street2: Optional[str] = Field(None, description="Street address line 2")
    city: str
    state: str
    zipCode: str


class Issuer(BaseModel):
    ticker: str = Field(description="Ticker of the issuer")
    name: str = Field(description="Name of the issuer")
    cik: str = Field(description="10-digit SEC CIK of the issuer")
    sic: Optional[str] = Field(
        None, description="Standard Industrial Classification code"
    )
    stateOfIncorporation: Optional[str]


class ReportingOwner(BaseModel):
    name: str = Field(description="Name of the reporting owner")
    cik: Optional[str] = Field(
        None, description="CIK of the reporting owner (if available)"
    )
    address: Address
    relationship: Optional[str] = Field(
        None, description="One of: Officer, Director, 10% owner, Affiliate, Other"
    )


class UnderlyingSecurity(BaseModel):
    title: str = Field(description="Title of the underlying security")
    conversionOrExercisePrice: Optional[float] = Field(None)
    conversionOrExerciseDate: Optional[str] = Field(
        None, description="Exercise date in ISO 8601 format (YYYY-MM-DD)"
    )
    underlyingSecurityShares: Optional[int] = Field(
        None, description="Number of shares underlying each derivative"
    )


class NonDerivativeTransaction(BaseModel):
    securityTitle: str = Field(description="Title of the security")
    transactionDate: str = Field(
        description="Transaction date in ISO 8601 format (YYYY-MM-DD)"
    )
    transactionCode: str = Field(
        description="Code indicating transaction type (A, D, P, S, etc.)"
    )
    transactionShares: float
    transactionPricePerShare: float
    totalTransactionValue: Optional[float] = Field(
        None, description="transactionShares × transactionPricePerShare"
    )
    ownershipNature: str = Field(description="e.g. Direct, Indirect")
    footnoteIds: Optional[List[str]] = Field(
        None, description="References into the footnotes block"
    )


class DerivativeTransaction(BaseModel):
    securityTitle: str
    transactionDate: str = Field(
        description="Transaction date in ISO 8601 format (YYYY-MM-DD)"
    )
    transactionCode: str
    numberOfDerivativeSecurities: int
    exercisePrice: Optional[float]
    expirationDate: Optional[str] = Field(
        description="Expiration date in ISO 8601 format (YYYY-MM-DD)"
    )
    underlyingSecurity: UnderlyingSecurity
    directOrIndirect: str = Field(description="Direct or Indirect")
    footnoteIds: Optional[List[str]]


class Footnote(BaseModel):
    id: str = Field(description="Footnote identifier")
    text: str = Field(description="Full footnote text")


class Form4Metadata(BaseModel):
    cik: str = Field(description="CIK of the filer")
    accessionNumber: str
    primaryDocument: str = Field(description="Filename of the primary XML/TXT")
    filingDate: str = Field(description="Filing date in ISO 8601 format (YYYY-MM-DD)")
    reportDate: str = Field(
        description="Period of report date in ISO 8601 format (YYYY-MM-DD)"
    )


class Form4(BaseModel):
    metadata: Form4Metadata
    issuer: Issuer
    reportingOwners: List[ReportingOwner]
    nonDerivativeTransactions: List[NonDerivativeTransaction]
    derivativeTransactions: List[DerivativeTransaction]
    footnotes: Optional[List[Footnote]] = Field(default=None)

### Creating the LlamaExtract Agent

In [ ]:
from dotenv import load_dotenv
from llama_cloud_services import LlamaExtract


# Load environment variables (put LLAMA_CLOUD_API_KEY in your .env file)
load_dotenv(override=True)

# Optionally, add your project id/organization id
extract = LlamaExtract(show_progress=False, check_interval=5)

In [ ]:
from llama_cloud.core.api_error import ApiError

try:
    existing_agent = extract.get_agent(name="Form4Extractor")
    if existing_agent:
        # Deletion can take some time since all underlying files will be purged
        extract.delete_agent(existing_agent.id)
except ApiError as e:
    if e.status_code == 404:
        pass
    else:
        raise

agent = extract.create_agent("Form4Extractor", data_schema=Form4)

### Example Extraction

We will write some utility functions to get the CIK mapping and the SEC Form 4 URLs for all the companies in the Dow Jones Industrial Average. 

⚠ Note that we need to limit ourselves to within 10reqs/second when hitting the SEC data so as not to get IP banned. We will use AsyncLimited for aiolimiter in our workflow for this. 

To do this, we will 
 - use `hishel` to cache the responses from the SEC website to avoid being rate limited or blocked, 
 - use `aiolimiter` to limit the rate of requests to the SEC website, and 
 - pre-populate a cache of the SEC Form 4 filings into a data dump that can be shared with a wider group.

In [ ]:
%pip install aiolimiter


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import json
from aiolimiter import AsyncLimiter
from collections import defaultdict


rate_limiter = AsyncLimiter(max_rate=5, time_period=1)


async def get_cik_mapping(tickers: List[str]) -> Dict[str, str]:
    if os.path.exists("./data/insider_transactions/cik_mapping.json"):
        with open("./data/insider_transactions/cik_mapping.json", "r") as f:
            return json.load(f)
    mapping_resp = await http_client.get(
        "https://www.sec.gov/files/company_tickers.json",
        headers=HEADERS,
    )
    mapping_resp.raise_for_status()
    mapping = mapping_resp.json()
    ticker_to_cik = {v["ticker"]: str(v["cik_str"]).zfill(10) for v in mapping.values()}
    with open("./data/insider_transactions/cik_mapping.json", "w") as f:
        json.dump(ticker_to_cik, f, indent=2)
    return {k: v for k, v in ticker_to_cik.items() if k in tickers}


async def get_sec_form4_urls(
    tickers: List[str], start_date: str, end_date: str
) -> List[str]:
    ticker_to_cik = await get_cik_mapping(tickers)
    urls = defaultdict(list)
    for ticker in tickers:
        cik = ticker_to_cik[ticker]
        url = f"https://data.sec.gov/submissions/CIK{cik}.json"
        async with rate_limiter:
            resp = await http_client.get(url)
            resp.raise_for_status()

        recent = resp.json().get("filings", {}).get("recent", {})
        forms = recent.get("form", [])
        dates = recent.get("filingDate", [])
        accession = recent.get("accessionNumber", [])
        for f, d, acc in zip(forms, dates, accession):
            if f == "4" and start_date <= d <= end_date:
                idx = acc.replace("-", "")
                urls[ticker].append(
                    f"https://www.sec.gov/Archives/edgar/data/{int(cik)}/{idx}/{acc}.txt",
                )
    return urls


async def get_sec_form4(url):
    form_4_cached = None
    if os.path.exists(FORM_4_DUMP_PATH):
        with open(FORM_4_DUMP_PATH, "r") as f:
            form_4_cached = json.load(f)
    accession_no = url.split("/")[-1].split(".")[0]
    if form_4_cached and accession_no in form_4_cached:
        return form_4_cached[accession_no]
    async with rate_limiter:
        resp = await http_client.get(url)
        resp.raise_for_status()
    return resp.text

In [ ]:
form_4_urls = await get_sec_form4_urls(["AAPL"], "2024-01-01", "2025-03-31")
form_4_urls

defaultdict(list,
            {'AAPL': ['https://www.sec.gov/Archives/edgar/data/320193/000032019325000036/0000320193-25-000036.txt',
              'https://www.sec.gov/Archives/edgar/data/320193/000032019325000035/0000320193-25-000035.txt',
              'https://www.sec.gov/Archives/edgar/data/320193/000032019325000034/0000320193-25-000034.txt',
              'https://www.sec.gov/Archives/edgar/data/320193/000032019325000033/0000320193-25-000033.txt',
              'https://www.sec.gov/Archives/edgar/data/320193/000032019325000032/0000320193-25-000032.txt',
              'https://www.sec.gov/Archives/edgar/data/320193/000032019325000031/0000320193-25-000031.txt',
              'https://www.sec.gov/Archives/edgar/data/320193/000032019325000030/0000320193-25-000030.txt',
              'https://www.sec.gov/Archives/edgar/data/320193/000032019325000022/0000320193-25-000022.txt',
              'https://www.sec.gov/Archives/edgar/data/320193/000032019325000021/0000320193-25-000021.txt',
  

In [ ]:
async def prepopulate_form_4_cache(tickers, start_date, end_date):
    form_4_dump = await http_client.get(
        "http://llamaindex-public-demo.s3.amazonaws.com/sec_form_4_dump.json"
    )
    with open(FORM_4_DUMP_PATH, "w") as f:
        f.write(form_4_dump.text)

    # Seeding for the first time:
    #
    # cache = {}
    # for ticker in tickers:
    #     form_4_urls = await get_sec_form4_urls(tickers, start_date, end_date)
    #     print(ticker)
    #     for url in form_4_urls[ticker]:
    #         form_4 = await get_sec_form4(url)
    #         cache[url.split("/")[-1].split(".")[0]] = form_4
    # with open(FORM_4_DUMP_PATH, "w") as f:
    #     json.dump(cache, f, indent=2)


START_DATE = "2024-01-01"
END_DATE = "2025-03-31"

await prepopulate_form_4_cache(DJIA_TICKERS, START_DATE, END_DATE)

In [ ]:
example_form_4 = await get_sec_form4(
    "https://www.sec.gov/Archives/edgar/data/320193/000032019325000036/0000320193-25-000036.txt"
)

In [ ]:
from llama_cloud_services.extract import SourceText

agent.extract(SourceText(text_content=example_form_4)).data

{'metadata': {'cik': '0001059235',
  'accessionNumber': '0000320193-25-000036',
  'primaryDocument': 'wk-form4_1740699336.xml',
  'filingDate': '2025-02-27',
  'reportDate': '2025-02-25'},
 'issuer': {'ticker': 'AAPL',
  'name': 'Apple Inc.',
  'cik': '0000320193',
  'sic': '3571',
  'stateOfIncorporation': 'CA'},
 'reportingOwners': [{'name': 'WAGNER SUSAN',
   'cik': '0001059235',
   'address': {'street1': 'ONE APPLE PARK WAY',
    'street2': None,
    'city': 'CUPERTINO',
    'state': 'CA',
    'zipCode': '95014'},
   'relationship': 'Director'}],
 'nonDerivativeTransactions': [],
 'derivativeTransactions': [{'securityTitle': 'Restricted Stock Unit',
   'transactionDate': '2025-02-25',
   'transactionCode': 'A',
   'numberOfDerivativeSecurities': 1255,
   'exercisePrice': None,
   'expirationDate': '2026-02-01',
   'underlyingSecurity': {'title': 'Common Stock',
    'conversionOrExercisePrice': None,
    'conversionOrExerciseDate': '2026-02-01',
    'underlyingSecurityShares': 1255}

## Workflow for Extracting Insider Transactions from SEC Form 4

Now that we have defined the LlamaExtract agent, we can create a workflow to scalably extract the insider transactions from SEC Form 4 filings for all the companies in the Dow Jones Industrial Average. We will use all the filings during 2024 and the first quarter of 2025.


### Exporting to Pandas DataFame for Analysis

Once we have the data extracted, we can create a Pandas DataFrame with the relevant subset of fields that we can use for analysis. In `forms_to_dataframe`, we will need to map the nested JSON data extracted using our schema above to a flat Pandas DataFrame for analysis.

**Workflow diagram:**

![Workflow Diagram](./data/insider_transactions/workflow-diag.png)


In [ ]:
from llama_index.core.workflow import (
    Context,
    Workflow,
    Event,
    StartEvent,
    StopEvent,
    step,
)
from llama_index.core.workflow.retry_policy import ConstantDelayRetryPolicy


class SECForm4URL(Event):
    "Form 4 URL to extract from"
    ticker: str
    url: str


class ExtractedData(Event):
    "Data extracted from SEC Form 4"
    extracted: dict


class ExtractedDataCollection(Event):
    "Collection of extracted data for all tickers"
    collection: List[dict]


class Form4Extraction(Workflow):
    def __init__(self, start_date: str, end_date: str, verbose=False, timeout=1800):
        super().__init__(verbose=verbose, timeout=timeout)
        self.start_date = start_date
        self.end_date = end_date
        self.limiter = AsyncLimiter(max_rate=9, time_period=1)
        self.result = None

    @step
    async def get_djia_companies(self, ctx: Context, ev: StartEvent) -> SECForm4URL:
        """
        Fetch all Form 4 URLs for companies in the DJIA index
        between (start_date, end_date).
        """
        url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
        parsed = pd.read_html(url, header=0)
        dow = next(t for t in parsed if "Symbol" in t.columns)
        tickers = dow["Symbol"].tolist()
        if self._verbose:
            print(f"Number of companies to extract: {len(tickers)}")
        result = []
        ticker_urls = await get_sec_form4_urls(tickers, self.start_date, self.end_date)
        for ticker, urls in ticker_urls.items():
            for url in urls:
                result.append(SECForm4URL(ticker=ticker, url=url))
        if self._verbose:
            print(f"Number of forms to extract: {len(result)}")
        await ctx.set("num_forms", len(result))
        for url in result:
            ctx.send_event(url)

    @step(
        num_workers=10,
        retry_policy=ConstantDelayRetryPolicy(maximum_attempts=3, delay=10),
    )
    async def fetch_sec_form4_filings(self, ev: SECForm4URL) -> ExtractedData:
        """
        Download form 4 from the SEC website, and use LlamaExtract agent defined earlier
        to extract the data from the form.
        """
        async with self.limiter:
            form_4 = await get_sec_form4(ev.url)
        extracted = await agent.aextract(
            SourceText(text_content=f"ticker: {ev.ticker}\n\nform 4: {form_4}")
        )
        return ExtractedData(extracted=extracted.data)

    @step
    async def collect_results(
        self, ctx: Context, ev: ExtractedData
    ) -> ExtractedDataCollection | None:
        """
        Collect all the data from the SEC forms into results.
        """
        num_forms = await ctx.get("num_forms")
        results = ctx.collect_events(ev, [ExtractedData] * num_forms)
        if results is None:
            return
        self.result = [r.extracted for r in results]
        if self._verbose:
            print(f"{len(self.result)} results extracted.")
        return ExtractedDataCollection(collection=self.result)

    @step
    async def forms_to_dataframe(
        self, ctx: Context, ev: ExtractedDataCollection
    ) -> StopEvent:
        """
        Convert the extracted data into a Pandas DataFrame.
        """
        # 1) map SEC transaction codes to human labels
        # Add any other codes you need to consider in your analysis...
        code_map = {
            "S": "Sale",
            "P": "Proposed Sale",
            "A": "Acquisition",
            "D": "Disposition",
        }
        records = []
        for form in ev.collection:
            filing_date = form["metadata"]["filingDate"]
            # loop each reporting owner (usually one)
            for owner in form.get("reportingOwners", []):
                # non-derivative transactions
                for tx in form.get("nonDerivativeTransactions", []) or []:
                    records.append(
                        {
                            "Ticker": form["issuer"]["ticker"],
                            "Company Name": form["issuer"]["name"],
                            "Insider Name": owner["name"],
                            "Relationship": owner.get("relationship"),
                            "Transaction Date": pd.to_datetime(tx["transactionDate"]),
                            "Transaction": code_map.get(
                                tx["transactionCode"], tx["transactionCode"]
                            ),
                            "Cost": tx["transactionPricePerShare"],
                            "#Shares": tx["transactionShares"],
                            "Value ($)": tx.get(
                                "totalTransactionValue",
                                tx["transactionShares"]
                                * tx["transactionPricePerShare"],
                            ),
                            "#Shares Total": tx.get("postTransactionShares"),
                            "Filing Date": pd.to_datetime(filing_date),
                        }
                    )

                # derivative transactions
                for tx in form.get("derivativeTransactions", []) or []:
                    records.append(
                        {
                            "Ticker": form["issuer"]["ticker"],
                            "Company Name": form["issuer"]["name"],
                            "Insider Name": owner["name"],
                            "Relationship": owner.get("relationship"),
                            "Transaction Date": pd.to_datetime(tx["transactionDate"]),
                            "Transaction": code_map.get(
                                tx["transactionCode"], tx["transactionCode"]
                            ),
                            "Cost": tx.get(
                                "transactionPricePerShare", tx.get("exercisePrice")
                            ),
                            "#Shares": tx.get("numberOfDerivativeSecurities"),
                            "Value ($)": None,
                            "#Shares Total": None,
                            "Filing Date": pd.to_datetime(filing_date),
                        }
                    )

        self.df = pd.DataFrame(records)
        self.df = self.df.sort_values(by=["Ticker", "Transaction Date"])
        return StopEvent()

In [ ]:
extraction = Form4Extraction(
    start_date="2025-03-01", end_date="2025-03-31", verbose=True
)
await extraction.run()

Running step get_djia_companies
Number of companies to extract: 30
Number of forms to extract: 259
Step get_djia_companies produced no event
Running step fetch_sec_form4_filings
Running step fetch_sec_form4_filings
Running step fetch_sec_form4_filings
Running step fetch_sec_form4_filings
Running step fetch_sec_form4_filings
Running step fetch_sec_form4_filings
Running step fetch_sec_form4_filings
Running step fetch_sec_form4_filings
Running step fetch_sec_form4_filings
Running step fetch_sec_form4_filings
Step fetch_sec_form4_filings produced event ExtractedData
Running step fetch_sec_form4_filings
Running step collect_results
Step collect_results produced no event
Step fetch_sec_form4_filings produced event ExtractedData
Running step fetch_sec_form4_filings
Running step collect_results
Step collect_results produced no event
Step fetch_sec_form4_filings produced event ExtractedData
Running step fetch_sec_form4_filings
Running step collect_results
Step collect_results produced no event


In [ ]:
extraction.df

,Ticker,Company Name,Insider Name,Relationship,Transaction Date,Transaction,Cost,#Shares,Value ($),#Shares Total,Filing Date
8,AMGN,AMGEN INC,Santos Esteban,Officer,2025-03-03,Acquisition,0.0000,6792.0,0.000000e+00,None,2025-03-05
9,AMGN,AMGEN INC,Miller Derek,Officer,2025-03-03,Acquisition,0.0000,2005.0,0.000000e+00,None,2025-03-05
10,AMGN,AMGEN INC,REESE DAVID M,Officer,2025-03-03,Acquisition,0.0000,7277.0,0.000000e+00,None,2025-03-05
13,AMGN,AMGEN INC,Griffith Peter H.,Officer,2025-03-03,Acquisition,0.0000,7277.0,0.000000e+00,None,2025-03-05
14,AMGN,AMGEN INC,Grygiel Nancy A.,Officer,2025-03-03,Acquisition,0.0000,1503.0,0.000000e+00,None,2025-03-05
...,...,...,...,...,...,...,...,...,...,...,...
446,WMT,Walmart Inc.,Penner Gregory Boyd,Director,2025-03-26,J,0.0000,27000.0,0.000000e+00,None,2025-03-28
457,WMT,Walmart Inc.,Walton Family Holdings Trust,10% owner,2025-03-26,J,0.0000,297000.0,0.000000e+00,None,2025-03-28
440,WMT,Walmart Inc.,McMillon C Douglas,"Officer, Director",2025-03-27,Sale,85.6333,29124.0,2.495473e+06,None,2025-03-31
445,WMT,Walmart Inc.,Walton Steuart L,Director,2025-03-28,G,0.0000,27000.0,0.000000e+00,None,2025-03-28
